In [1]:
#imports

import io
import numpy as np

In [2]:
pip install pyxlsb

Note: you may need to restart the kernel to use updated packages.


## == LOAD WORD EMBEDDINGS == 

In [3]:
# Our embeddings from FastText have 200.000 embeddings per language.

def load_vec(emb_path, nmax= 200000):
    vectors = []
    word2id = {}
    with io.open(emb_path, 'r', encoding='utf-8', newline='\n', errors='ignore') as f:
        next(f)
        for i, line in enumerate(f):
            word, vect = line.rstrip().split(' ', 1)
            vect = np.fromstring(vect, sep=' ')
            assert word not in word2id, 'word found twice'
            vectors.append(vect)
            word2id[word] = len(word2id)
            if len(word2id) == nmax:
                break
    id2word = {v: k for k, v in word2id.items()}
    embeddings = np.vstack(vectors)
    return embeddings, id2word, word2id

In [4]:
# Load embeddings. We will work with 3 languages: English, Spanish and French.

# These are the path routes with the embeddings in each language
english_path = r'C:\Users\maria\Desktop\AIMASTERS\INTERNSHIP\wiki.multi.en.vec.txt'
spanish_path = r'C:\Users\maria\Desktop\AIMASTERS\INTERNSHIP\wiki.multi.es.vec.txt'
french_path = r'C:\Users\maria\Desktop\AIMASTERS\INTERNSHIP\wiki.multi.fr.vec.txt'

nmax = 200000 # maximum number of word embeddings to load

english_embeddings, english_id2word, english_word2id = load_vec(english_path, nmax)
spanish_embeddings, spanish_id2word, spanish_word2id = load_vec(spanish_path, nmax)
french_embeddings, french_id2word, french_word2id = load_vec(french_path, nmax)

In [5]:
metadata_filename = "metadataa.tsv"
tensor_filename = "tensora.tsv"

## == Add a language mark to each word to clarify the language it comes from. == 

In [6]:
# This was done because some words were included in 2 of the files. For example, the word sandwich was found in the
# spanish and english file so we added sandiwich_en and sandwich_es to differenciate them.

english_vocab = [element[1]+ "_en" for element in english_id2word.items()]
spanish_vocab = [element[1]+ "_es" for element in spanish_id2word.items()]
french_vocab = [element[1]+ "_fr" for element in french_id2word.items()]


print(english_vocab[0:5])
print(spanish_vocab[0:5])
print(french_vocab[0:5])

[',_en', '._en', 'the_en', '</s>_en', 'of_en']
['de_es', '</s>_es', ',_es', '._es', 'la_es']
['</s>_fr', ',_fr', 'de_fr', "'_fr", '._fr']


## == Get the tensors for each language in lists ==

In [7]:
def tensor_for_label(embedding):
    """ Transform the embedding list to a tuple"""
    return tuple(embedding)

In [8]:
tensors_english = [tensor_for_label(embedding) for embedding in english_embeddings]
tensors_spanish = [tensor_for_label(embedding) for embedding in spanish_embeddings]
tensors_french = [tensor_for_label(embedding) for embedding in french_embeddings]

## Create a dictionary word & tensor for every language and concatenate the 3 dictionaries!

In [9]:
# ZIP a word with its tensor of embeddings:
english_dic = dict(zip(english_vocab, tensors_english))
spanish_dic = dict(zip(spanish_vocab, tensors_spanish))
french_dic = dict(zip(french_vocab, tensors_french))

In [10]:
# english_dic.keys()

In [11]:
spanish_dic.update(english_dic)
spanish_dic.update(french_dic)

In [12]:
print(f" In total we have {len(spanish_dic)} embeddings which contain Spanish, English and French words")

 In total we have 600000 embeddings which contain Spanish, English and French words


## == Nearest neighbours of a word in the other languages == 

In [13]:
# src = source language, tgt = target language in which we want to look at the NN.

def get_nn(word, src_emb, src_id2word, tgt_emb, tgt_id2word, K=200000):
    knn_list = []
    word2id = {v: k for k, v in src_id2word.items()}
    #print(word, word2id)
    word_emb = src_emb[word2id[word]]
    scores = (tgt_emb / np.linalg.norm(tgt_emb, 2, 1)[:, None]).dot(word_emb / np.linalg.norm(word_emb))
    k_best = scores.argsort()[-K:][::-1]
    for i, idx in enumerate(k_best):
        #print('%.4f - %s' % (scores[idx], tgt_id2word[idx]))
        knn_list.append((scores[idx], tgt_id2word[idx]))
    return word, knn_list

In [14]:
# Look at the word fencing which is source = English and check the NN in target language = spanish. 
# K is the number of NN we want.
get_nn("fencing", english_embeddings, english_id2word, spanish_embeddings, spanish_id2word, K=5)

('fencing',
 [(0.5886008348392442, 'atletismo'),
  (0.5721339380527912, 'equitación'),
  (0.5713304801722254, 'judo'),
  (0.5616311101795585, 'esgrima'),
  (0.5413254101786309, 'natación')])

## == EVALUATION OF POLYSEMIC WORDS AND COMMON vs UNCOMMON == 

In [15]:
# The following words were selected from specific databases. (Explained in the pdf)

In [16]:
# polysemic words
polysemic_words_english = ['star', 'letter', 'wing', 'date', "head", "radio", 'right', 'crane', 'area', 'period', 'character', 'paper', 'fan', 'bank', 'mouse', 'buck', 'chip', 'arm', 'record', 'cell', 'light', 'blue', 'foot', 'office', 'bug', 'double', 'habit', 'infant', 'serie', 'icon']
polysemic_words_spanish = ["banco", "alianza", "cubo", "carta", "paleta",  "órgano", "regla", "salsa", "bolsa", "cola", "vela", "raton", "tiempo", "tierra", "copa", "hoja", "estrellas", "lengua",  "planta", 'arco', 'fuente', 'muñeca', 'batería', 'estacion', 'cura', 'mono',  'tarde', 'banda', 'metro']
polysemic_words_french = ["ampoule", "point", "ensemble", "sens", "suite", "baguette", "kiwi", "chemise", "carte", "tableau", "bouton", "avocat", "jumelles", "glace", "souris", "sirène", "feuille", "terre", "loup", "planche", "lettre", "tête", "nature", "face", "angle", "coffre", "rayon", "balle", "service"]

# not that common words in English:
unfrequent_words_english = ["flesh", "nest", "reluctant", "wax", "quantum", "reformer", "upstairs", "spouse", "urine", "slogan", "solvent", "pest", "screw", "whisper", "twist", "meadow", "toll", "sanctuary", "dome", "royalty", "neglect", "ivory", "fool", "velvet", "banner", "telly", "whip", "weed", "pioneer", "glow", "gossip"]

# Common words in English:
frequent_words_english = ["her", "year", "cat", "dog", "doctor", "nurse", "president", "people", "day", "thing", "child", "system", "group", "company", "problem", "party", "hand", "place", "house", "week", "word", "family", "home", "question", "car", "idea", "name", "body", "door", "girl", "table", "church", "student", "morning", "game", "boy", "teacher", "window", "project", "tax", "nature", "method", "bed"]

# not that common words in Spanish:
unfrequent_words_spanish = [ "oleaje", "opositor", "dramaturgo", "zurdo", "inferencia", "colonizar", "indefinida", "safari", "colchoneta", "codo", "inciso",  "clavel", "epitafio",  "pixel",  "anarquista", "encierro", "peine", "enchufe", "ahogo"]
# common words in Spanish:
frequent_words_spanish = ["dios", "oficina", "contrato", "modelo", "misterio", "vacaciones", "minuto", "minimo", "millon", "mil", "uno", "mierda", "miembro", "miedo", "mes", "mente", "comer", "salud", "sana",  "color", "colegio", "sabado"]

# not that common words in French:
unfrequent_words_french = [ "pressage", "libretto", "antiracisme", "antiterrorisme", "effluent", "hippisme", "tromboniste", "portfolio", "histamine", "liquor", "traveller",  "rupteur", "transplant", "balboa", "fluor", "quiz",  "ratio", "robotique", "rational", "puffin"]

# Common words in French:
frequent_words_french = ["homme", "jour", "main", "femme", "vie", "enfant", "moment", "fille", "maison", "mot", "bras", "eau", "amour", "matin", "nom", "guerre", "ville", "silence", "train", "tour", "famille", "mer", "couleur", "sentiment", "photo", "cousine", "professeur", "professeure", "violence", "oeuf", "auto" ]

In [17]:
# Sort in alphabetic order 

polysemic_words_english = sorted(polysemic_words_english)
polysemic_words_spansih = sorted(polysemic_words_spanish)
polysemic_words_french = sorted(polysemic_words_french)

unfrequent_words_english = sorted(unfrequent_words_english)
frequent_words_english = sorted(frequent_words_english)

unfrequent_words_spanish  = sorted(unfrequent_words_spanish )
frequent_words_spanish = sorted(frequent_words_spanish)

frequent_words_french = sorted(frequent_words_french)
unfrequent_words_french = sorted(unfrequent_words_french)

## == Polysemic English Words == 

In [18]:
import pandas as pd
df_english = pd.DataFrame(columns=['WORD', 'language', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
languages_en = [(spanish_embeddings, spanish_id2word, "spanish"), (french_embeddings, french_id2word, "french")]


for word in polysemic_words_english:
    for tuple_languages in languages_en:
        word, lista = get_nn(word, english_embeddings, english_id2word, tuple_languages[0], tuple_languages[1], K=10)
        knn = [word[1] for word in lista]
        df_english  = df_english .append({'WORD': word, 'language':tuple_languages[2],  '1': knn[0], '2': knn[1], '3': knn[2], '4': knn[3], '5': knn[4], '6': knn[5], '7': knn[6], '8':knn[7], '9': knn[8], '10': knn[9]}, ignore_index = True)    


In [19]:
for element in df_english:
    df_english[element] = df_english[element].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [20]:
#df_english.to_csv("evaluation_english_last.csv", index=False, encoding = "utf-8" )

## == Polysemic Spanish Words == 

In [21]:
import pandas as pd
df_spanish = pd.DataFrame(columns=['WORD', 'language', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
languages = [(english_embeddings, english_id2word, "english"), (french_embeddings, french_id2word, "french")]


for word in polysemic_words_spansih:
    for tuple_languages in languages:
        word, lista = get_nn(word, spanish_embeddings, spanish_id2word, tuple_languages[0], tuple_languages[1], K=10)
        knn = [word[1] for word in lista]
        df_spanish  = df_spanish.append({'WORD': word, 'language':tuple_languages[2],  '1': knn[0], '2': knn[1], '3': knn[2], '4': knn[3], '5': knn[4], '6': knn[5], '7': knn[6], '8':knn[7], '9': knn[8], '10': knn[9]}, ignore_index = True)    


In [22]:
for element in df_spanish:
    df_spanish[element] = df_spanish[element].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [23]:
# df_spanish.to_csv("evaluation_spanish_last.csv", index=False, encoding = "utf-8" )

## == Polysemic French Words == 

In [24]:
import pandas as pd
df_french = pd.DataFrame(columns=['WORD', 'language', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
languages_ = [(english_embeddings, english_id2word, "english"), (spanish_embeddings, spanish_id2word, "spanish")]


for word in polysemic_words_french:
    for tuple_languages in languages_:
        word, lista = get_nn(word, french_embeddings, french_id2word, tuple_languages[0], tuple_languages[1], K=10)
        knn = [word[1] for word in lista]
        df_french  = df_french.append({'WORD': word, 'language':tuple_languages[2],  '1': knn[0], '2': knn[1], '3': knn[2], '4': knn[3], '5': knn[4], '6': knn[5], '7': knn[6], '8':knn[7], '9': knn[8], '10': knn[9]}, ignore_index = True)    


In [25]:
for element in df_french:
    df_french[element] = df_french[element].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [26]:
#df_french.to_csv("evaluation_french_final.csv", index=False, encoding = "utf-8" )

## == Uncommon and common English words ==

In [27]:
import pandas as pd
df_english_uncommon = pd.DataFrame(columns=['WORD', 'language', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
languages_en = [(spanish_embeddings, spanish_id2word, "spanish"), (french_embeddings, french_id2word, "french")]

for word in unfrequent_words_english:
    for tuple_languages in languages_en:
        word, lista = get_nn(word, english_embeddings, english_id2word, tuple_languages[0], tuple_languages[1], K=10)
        knn = [word[1] for word in lista]
        df_english_uncommon  = df_english_uncommon.append({'WORD': word, 'language':tuple_languages[2],  '1': knn[0], '2': knn[1], '3': knn[2], '4': knn[3], '5': knn[4], '6': knn[5], '7': knn[6], '8':knn[7], '9': knn[8], '10': knn[9]}, ignore_index = True)    


In [28]:
for element in df_english_uncommon:
    df_english_uncommon[element] = df_english_uncommon[element].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [29]:
#df_english_uncommon.to_csv("evaluation_english_uncommon_final.csv", index=False, encoding = "utf-8" )

## == Uncommon and common Spanish words ==

In [30]:
import pandas as pd
df_spanish_common = pd.DataFrame(columns=['WORD', 'language', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
languages_es_common = [(english_embeddings, english_id2word, "english"), (french_embeddings, french_id2word, "french")]

for word in frequent_words_spanish:
    for tuple_languages in languages_es_common:
        word, lista = get_nn(word, spanish_embeddings, spanish_id2word, tuple_languages[0], tuple_languages[1], K=10)
        knn = [word[1] for word in lista]
        df_spanish_common = df_spanish_common.append({'WORD': word, 'language':tuple_languages[2],  '1': knn[0], '2': knn[1], '3': knn[2], '4': knn[3], '5': knn[4], '6': knn[5], '7': knn[6], '8':knn[7], '9': knn[8], '10': knn[9]}, ignore_index = True)    


In [31]:
for element in df_spanish_common:
    df_spanish_common[element] = df_spanish_common[element].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [32]:
# df_spanish_common.to_csv("df_spanish_common_final.csv", index=False, encoding = "utf-8" )

In [33]:
import pandas as pd
df_spanish_uncommon = pd.DataFrame(columns=['WORD', 'language', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
languages_es_uncommon = [(english_embeddings, english_id2word, "english"), (french_embeddings, french_id2word, "french")]

for word in unfrequent_words_spanish:
    for tuple_languages in languages_es_common:
        word, lista = get_nn(word, spanish_embeddings, spanish_id2word, tuple_languages[0], tuple_languages[1], K=10)
        knn = [word[1] for word in lista]
        df_spanish_uncommon = df_spanish_uncommon.append({'WORD': word, 'language':tuple_languages[2],  '1': knn[0], '2': knn[1], '3': knn[2], '4': knn[3], '5': knn[4], '6': knn[5], '7': knn[6], '8':knn[7], '9': knn[8], '10': knn[9]}, ignore_index = True)    


In [34]:
for element in df_spanish_uncommon:
    df_spanish_uncommon[element] = df_spanish_uncommon[element].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [35]:
# df_spanish_uncommon.to_csv("df_spanish_uncommon_final.csv", index=False, encoding = "utf-8" )

## == Uncommon and common French words ==

In [36]:
import pandas as pd
df_french_common = pd.DataFrame(columns=['WORD', 'language', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
languages_fr_common = [(english_embeddings, english_id2word, "english"), (spanish_embeddings, spanish_id2word, "spanish")]

for word in frequent_words_french:
    for tuple_languages in languages_fr_common:
        word, lista = get_nn(word, french_embeddings, french_id2word, tuple_languages[0], tuple_languages[1], K=10)
        knn = [word[1] for word in lista]
        df_french_common = df_french_common.append({'WORD': word, 'language':tuple_languages[2],  '1': knn[0], '2': knn[1], '3': knn[2], '4': knn[3], '5': knn[4], '6': knn[5], '7': knn[6], '8':knn[7], '9': knn[8], '10': knn[9]}, ignore_index = True)    


In [37]:
for element in df_french_common:
    df_french_common[element] = df_french_common[element].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [38]:
# df_french_common.to_csv("df_french_common_final.csv", index=False, encoding = "utf-8" )

In [39]:
import pandas as pd
df_french_uncommon = pd.DataFrame(columns=['WORD', 'language', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
languages_fr_uncommon = [(english_embeddings, english_id2word, "english"), (spanish_embeddings, spanish_id2word, "spanish")]

for word in unfrequent_words_french:
    for tuple_languages in languages_fr_uncommon:
        word, lista = get_nn(word, french_embeddings, french_id2word, tuple_languages[0], tuple_languages[1], K=10)
        knn = [word[1] for word in lista]
        df_french_uncommon = df_french_uncommon.append({'WORD': word, 'language':tuple_languages[2],  '1': knn[0], '2': knn[1], '3': knn[2], '4': knn[3], '5': knn[4], '6': knn[5], '7': knn[6], '8':knn[7], '9': knn[8], '10': knn[9]}, ignore_index = True)    


In [40]:
for element in df_french_uncommon:
    df_french_uncommon[element] = df_french_uncommon[element].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [41]:
# df_french_uncommon.to_csv("df_french_uncommon_final.csv", index=False, encoding = "utf-8" )